In [526]:
# Feature Analysis flow
from sklearn import svm
from sklearn.feature_selection import RFE
X = [[0, 0], [2, 2]]
y = [0.5, 2.5]
regr = svm.SVR()
selector = RFE(regr, n_features_to_select=5, step=1)
selector = selector.fit(X, y)
selector.support_
selector.ranking_


array([1, 1])

In [527]:
# load data
import numpy as np
import pandas as pd
df = pd.read_excel('OuterHarbour-Tiel.xlsx')
# df = pd.read_excel('OuterHarbour-Tiel2.xlsx')

In [528]:
# All nodes on the route
locs = df[["locationType", "cityName", "countryName"]].to_numpy().astype('str')
nodes = np.unique(locs, axis=0)
# create columns with default value for now
for node in nodes:
    df[f'{node[0]}_{node[1]}_{node[2]}_visited'] = False
    df[f'{node[0]}_{node[1]}_{node[2]}_planned'] = False

In [529]:
df = df[df["ordSeaFreightMainTransportInvolvement"] == "J"]
df["ordJFHArrangeLoadingPickup"] = df["ordJFHArrangeLoadingPickup"].fillna("")
df["ordJFHArrangeDelivery"] = df["ordJFHArrangeDelivery"].fillna("")
# df["ordReqDeliveryDate"] = df["ordReqDeliveryDate"].fillna("")

df.drop_duplicates(subset=['rowNo'], keep='first')

,shipmentID,rowNo,originDestination,locationType,readyDate,cityName,countryName,estimatedArrDate,actualArrDate,estimatedDepDate,...,P1_Marsaxlokk_Malta_visited,P1_Marsaxlokk_Malta_planned,P1_Tekirdag_Turkey_visited,P1_Tekirdag_Turkey_planned,P1_Valencia_Spain_visited,P1_Valencia_Spain_planned,P2_Antwerpen_Belgium_visited,P2_Antwerpen_Belgium_planned,P2_Valencia_Spain_visited,P2_Valencia_Spain_planned
0,3334658,1,O,A,2021-01-07,Outer Harbour,Australia,NaT,NaT,2021-01-14,...,False,False,False,False,False,False,False,False,False,False
1,3334658,2,O,H1,NaT,Outer Harbor,Australia,2021-02-25 00:00:00,2021-02-25 00:00:00,2021-03-02,...,False,False,False,False,False,False,False,False,False,False
2,3334658,3,O,P,NaT,Adelaide,Australia,2021-03-02 14:00:00,2021-03-01 20:38:00,2021-03-04,...,False,False,False,False,False,False,False,False,False,False
3,3334658,4,D,P,NaT,Rotterdam,Netherlands,2021-04-18 00:00:00,2021-04-18 12:00:00,NaT,...,False,False,False,False,False,False,False,False,False,False
4,3334658,5,D,D,NaT,Maasvlakte,Netherlands,NaT,NaT,NaT,...,False,False,False,False,False,False,False,False,False,False
5,3334658,6,D,A,NaT,Tiel,Netherlands,2021-04-20 06:00:00,2021-04-20 06:00:00,NaT,...,False,False,False,False,False,False,False,False,False,False
24,3458782,7,D,D,NaT,Maasvlakte,Netherlands,NaT,NaT,NaT,...,False,False,False,False,False,False,False,False,False,False
25,3458782,8,D,A,NaT,Tiel,Netherlands,2021-06-02 08:00:00,2021-06-02 08:00:00,NaT,...,False,False,False,False,False,False,False,False,False,False


In [530]:
from math import sqrt, pow
def node_name(node):
    # node = node.iloc[0]
    return f'{node["locationType"]}_{node["cityName"]}_{node["countryName"]}'

def abs_distance(o, lat, long):
    # o = origin_node.iloc[0]
    diff_lat = abs(o["latitude"] - lat) * 111.139
    diff_long =  abs(o["longitude"] - long) * 111.139
    return sqrt(pow(diff_lat, 2) + pow(diff_long, 2))

def dep_late(node):
    return node["actualDepDate"] > node["estimatedDepDate"]

df["origin"] = [df.loc[(df["shipmentID"] == shipmentID) & (df["rowNo"] == 1)].iloc[0] for shipmentID in df["shipmentID"]]
df["originCountry"] = [o["countryName"] for o in df["origin"]]
df["destination"] = [df.loc[(df["shipmentID"] == shipmentID) & (df["rowNo"] == df[df["shipmentID"] == shipmentID]["rowNo"].max())].iloc[0] for shipmentID in df["shipmentID"]]
df["destinationCountry"] = [d["countryName"] for d in df["destination"]]
df["readyDate"] = [o["readyDate"] for o in df["origin"]]
df["consolidation"] = (df["ordtType"].str.strip() == "LF") | (df["ordtType"].str.strip() == "LFL")
df["deconsolidation"] = df["ordtType"].str.strip() == "LFL"
df["serviceType"] = [f"{'door' if p == 'Y' else 'port'}-to-{'door' if d == 'Y' else 'port'}" for p, d in df[["ordJFHArrangeLoadingPickup", "ordJFHArrangeDelivery"]].values]
df["pastTimeToOrigin"] = [orig["actualDepDate"] - arr_date for orig, arr_date in df[["origin", "actualArrDate"]].values]
df["pastAvgAbsSpeed"] = [past_time / abs_distance(orig, lat, long) for (lat, long, orig, past_time) in df[["latitude", "longitude", "origin", "pastTimeToOrigin"]].values]
df["counterStops"] = df["rowNo"] - 1
df["departureMonthOrigin"] = 0 # need row created date for this
df["departureDayOrigin"] = 0 # need row created date for this
df["numberOfTEU"] = df["ordExpectedNBOfContainers"]
df["totalWeight"] = df["ordGrossWeightKGM"]
# df["ETA"] = df["ordReqDeliveryDate"]
df["ATA"] = [df.loc[(df["shipmentID"] == shipmentID) & (df["rowNo"] == df[df["shipmentID"] == shipmentID]["rowNo"].max())].iloc[0]["actualArrDate"] for shipmentID in df["shipmentID"]]
df["currentMonth"] = 0 # need row created date for this
df["portOfLoading"] = "unknown" # need row created date for this
df["portOfDischarge"] = "unknown" # need row created date for this
df["portDestCongestion"] = None # need row created date for this
# TODO: add check for current date > act. departure date -- need row created date for this
df["portOfLoadingLate"] = [dep_late(df.loc[(df["shipmentID"] == shipmentID) & (df["locationType"].str.strip() == "P") & (df["originDestination"].str.strip() == "O")].iloc[0]) for shipmentID in df["shipmentID"]]
df["origin"] = [node_name(o) for o in df["origin"]]
df["destination"] = [node_name(d) for d in df["destination"]]
# TODO: set visited path

# TODO: set future path


# df = df.drop(columns=["latitude" , "longitude", "actualArrDate", "ordExpectedNBOfContainers", "ordGrossWeightKGM", "rowNo", "shipmentID", "originDestination", "locationType", "cityName", "countryName", "ordtType", "ordJFHArrangeLoadingPickup", "ordSeaFreightMainTransportInvolvement", "ordJFHArrangeDelivery"])

# df.head(10)

In [539]:
ss = np.unique(df["shipmentID"])
paths = [df[df["shipmentID"] == s][["locationType", "cityName", "countryName"]].apply(lambda x : node_name(x), axis=1) for s in ss]
for path, shipment in zip(paths, ss):
    for i in range(1, len(path) + 1):
        df.loc[(df["shipmentID"] == shipment) & (df["rowNo"] == i), [f"{n}_visited" for n in path[:i]]] = True
        df.loc[(df["shipmentID"] == shipment) & (df["rowNo"] == i), [f"{n}_planned" for n in path[i-1:]]] = True

# df = df.drop(columns=["latitude" , "longitude", "actualArrDate", "ordExpectedNBOfC ontainers", "ordGrossWeightKGM", "rowNo", "shipmentID", "originDestination", "locationType", "cityName", "countryName", "ordtType", "ordJFHArrangeLoadingPickup", "ordSeaFreightMainTransportInvolvement", "ordJFHArrangeDelivery"])
df.to_excel("output.xlsx")